In [5]:
# DASH Framework for Jupyter
# URL Lib to make sure that our input is 'sane'
import urllib.parse
from typing import Optional

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import grazioso  # Import your CRUD Python module

# Build App
app = dash.Dash(__name__)
app.layout = html.Div([
    # This element generates an HTML Heading with your name
    html.H1("Module 5 Assignment - Brett Plemons"),
    # This Input statement sets up an Input field for the username.
    dcc.Input(
            id="input_user".format(),
            type="text",
            placeholder="input type {}".format("text")),
    # This Input statement sets up an Input field for the password.
    # This designation masks the user input on the screen.
    dcc.Input(
            id="input_passwd".format(),
            type="password",
            placeholder="input type {}".format("password")),
    # Create a button labeled 'Submit'. When the button is pressed
    # the n_clicks value will increment by 1. 
    html.Button('Submit', id='submit-val', n_clicks=0),
    # Generate a horizontal line separating our input from our
    # output element
    html.Hr(),
    # This sets up the output element for the dashboard. The
    # purpose of the stlye option is to make sure that the 
    # output will function like a regular text area and accept
    # newline ('\n') characters as line-breaks.
    html.Div(id="query-out", style={'whiteSpace': 'pre-line'}),
    html.H3("Student ID: 2498972 - MongoDB Authentication Dashboard")
])

# Define callback to update output-block
# NOTE: While the name of the callback function doesn't matter,
# the order of the parameters in the callback function are the
# same as the order of Input methods in the @app.callback
# For the callback function below, the callback is grabing the
# information from the input_user and input_password entries, and
# then the value of the submit button (has it been pressed?)
@app.callback(
    Output('query-out', 'children'),
    [Input('input_user', 'value'),
     Input('input_passwd', 'value'),
     Input('submit-val', 'n_clicks')]
)
def update_figure(input_user: Optional[str], input_passwd: Optional[str], n_clicks: int) -> str:
    # This is used as a trigger to make sure that the callback doesn't
    # try and connect to the database until after the submit button
    # is pressed. Otherwise, every time a character was added to the 
    # username or password field, an attempt would be made to connect to 
    # the database with an incorrect username and password.
    if n_clicks > 0:
        ###########################
        # Data Manipulation / Model
        # use CRUD module to access MongoDB
        ##########################
        
        # Use urllib.parse.quote_plus to properly encode username and password for MongoDB URI
        username = urllib.parse.quote_plus(str(input_user)) if input_user is not None else None 
        password = urllib.parse.quote_plus(str(input_passwd)) if input_passwd is not None else None
        
        ## DEBUG STATEMENT - You can uncomment the next line to verify you
        ## are correctly entering your username and password prior to continuing
        ## to build the callback function.
        ## return f'Output: {inputUser}, {inputPass}'

        #TODO: Instantiate CRUD object with above authentication username and 
        # password values
        shelter = grazioso.AnimalShelter(username, password)  # Instantiate your CRUD object

        #TODO: Return example query results. Note: The results returned have
        # to be in the format of a string in order to display properly in the 
        # 'query-out' element. Please separate each result with a newline for
        # readability
        
        # Run the specified query for a dog named Lucy
        query = {"animal_type": "Dog", "name": "Lucy"}
        results = shelter.read(query)
        
        # Convert results to string format for display
        output_string = ""
        for document in results:
            output_string += str(document) + "\n\n"
            
        return output_string
    
    # If the button hasn't been clicked, don't display anything
    return ""

# Run app and display result inline in the notebook
if __name__ == "__main__":
    app.run(debug=True)